# Options Sentiment & Volatility Regime Detection

This notebook introduces an options-based volatility sentiment proxy
using India VIX to enhance market regime detection.

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load spot feature data
spot_df = pd.read_csv("../data/processed/spot_features_1y.csv")

spot_df["date"] = pd.to_datetime(spot_df["date"])
spot_df = spot_df.sort_values("date").reset_index(drop=True)

spot_df.head()

,date,close,high,low,open,volume,ema_fast,ema_slow,ema_spread,trend_dir,log_return,rolling_vol,high_vol,market_regime
0,2021-10-21 15:25:00+05:30,18219.60,18222.35,18198.40,18198.40,0,18219.600000,18219.600000,0.000000,-1,NaN,NaN,0,bear_trend_low_vol
1,2021-10-22 09:15:00+05:30,18249.90,18251.00,18206.15,18230.70,0,18222.485714,18220.788235,1.697479,1,0.001662,NaN,0,bull_trend_low_vol
2,2021-10-22 09:20:00+05:30,18248.25,18276.00,18225.95,18251.00,0,18224.939456,18221.865167,3.074289,1,-0.000090,NaN,0,bull_trend_low_vol
3,2021-10-22 09:25:00+05:30,18279.15,18283.15,18248.50,18248.50,0,18230.102365,18224.111631,5.990733,1,0.001692,NaN,0,bull_trend_low_vol
4,2021-10-22 09:30:00+05:30,18295.40,18295.60,18273.85,18280.05,0,18236.321187,18226.907254,9.413934,1,0.000889,NaN,0,bull_trend_low_vol


In [18]:
vix_df = pd.read_csv("../data/raw/india_vix.csv")
vix_df.head()

,Date,Open,High,Low,Close,Prev. Close,Change,% Change
0,15-JAN-2025,15.4700,15.6625,15.1975,15.2575,15.4700,-0.21,-1.37
1,16-JAN-2025,15.2575,15.5725,13.9575,15.4650,15.2575,0.21,1.36
2,17-JAN-2025,15.4650,16.3825,14.0525,15.7475,15.4650,0.28,1.83
3,20-JAN-2025,15.7475,17.0200,15.1825,16.4150,15.7475,0.67,4.24
4,21-JAN-2025,16.4150,17.4450,15.2025,17.0550,16.4150,0.64,3.90


In [19]:
vix_df.columns

Index(['Date ', 'Open ', 'High ', 'Low ', 'Close ', 'Prev. Close ', 'Change ',
       '% Change '],
      dtype='object')

In [20]:
# Normalize column names
vix_df.columns = [c.lower().strip() for c in vix_df.columns]

vix_df.columns

Index(['date', 'open', 'high', 'low', 'close', 'prev. close', 'change',
       '% change'],
      dtype='object')

In [21]:
vix_df = vix_df.rename(columns={"index date": "date"})

In [22]:
vix_df["date"] = pd.to_datetime(vix_df["date"])

vix_df = vix_df[["date", "close"]].rename(columns={"close": "india_vix"})

vix_df.head()


C:\Users\Acer\AppData\Local\Temp\ipykernel_3064\2957032782.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  vix_df["date"] = pd.to_datetime(vix_df["date"])


,date,india_vix
0,2025-01-15,15.2575
1,2025-01-16,15.4650
2,2025-01-17,15.7475
3,2025-01-20,16.4150
4,2025-01-21,17.0550


In [23]:
spot_df = spot_df.sort_values("date").reset_index(drop=True)
vix_df = vix_df.sort_values("date").reset_index(drop=True)

In [24]:
spot_df["date"] = spot_df["date"].dt.tz_localize(None)

In [25]:
vix_df["date"] = pd.to_datetime(vix_df["date"]).dt.tz_localize(None)

In [26]:
spot_df = spot_df.sort_values("date").reset_index(drop=True)
vix_df = vix_df.sort_values("date").reset_index(drop=True)

In [27]:
spot_vix_df = pd.merge_asof(
    spot_df,
    vix_df,
    on="date",
    direction="backward"
)

spot_vix_df[["date", "india_vix"]].head()

,date,india_vix
0,2021-10-21 15:25:00,NaN
1,2021-10-22 09:15:00,NaN
2,2021-10-22 09:20:00,NaN
3,2021-10-22 09:25:00,NaN
4,2021-10-22 09:30:00,NaN


In [28]:
spot_vix_df["vix_ma"] = spot_vix_df["india_vix"].rolling(20).mean()
spot_vix_df["vix_high"] = (spot_vix_df["india_vix"] > spot_vix_df["vix_ma"]).astype(int)

In [29]:
spot_vix_df.to_csv(
    "../data/processed/spot_features_vix_1y.csv",
    index=False
)

In [30]:
# Create rolling mean of VIX
spot_vix_df["vix_ma"] = spot_vix_df["india_vix"].rolling(20).mean()

# Volatility regime: 1 = high vol, 0 = normal
spot_vix_df["vix_high"] = (spot_vix_df["india_vix"] > spot_vix_df["vix_ma"]).astype(int)

spot_vix_df[["date", "india_vix", "vix_ma", "vix_high"]].dropna().head()

,date,india_vix,vix_ma,vix_high


In [31]:
output_path = "../data/processed/spot_features_vix_1y.csv"
spot_vix_df.to_csv(output_path, index=False)

print("Enhanced spot features with VIX saved at:", output_path)

Enhanced spot features with VIX saved at: ../data/processed/spot_features_vix_1y.csv


## Summary

This notebook extends the baseline regime detection by incorporating
an options-derived volatility proxy (India VIX).

The VIX-based regime helps distinguish risk-on and risk-off market
conditions, improving robustness when combining with trend and ML filters.